# 텐서플로 2 시작 - 초보용

Keras를 사용하여 다음을 수행합니다.

1. 사전에 빌드한 데이터세트를 로드합니다.
2. 이미지를 분류하는 신경망 머신 러닝 모델을 빌드합니다.
3. 이 신경망을 훈련합니다.
4. 모델의 정확도를 평가합니다.

## Tensorflow 설정

Tensorflow 프로그램 가져옴

GPU를 사용하려면 NVIDIA 제어판에서 3D 설정관리 > 전역 설정 > 고성능 NVIDIA 프로세서로 선택, 확인 후 진행필요

In [1]:
import tensorflow as tf

f'Tensorflow version : {tf.__version__}'

'Tensorflow version : 2.18.0'

In [2]:
from tensorflow.python.client import device_lib

# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8837507231814329979
 xla_global_id: -1]

## 데이터세트 로드

MNIST 데이테세트. 이미지의 픽셀 값 범위는 0~255. 이때 값을 255.0으로 나누어 0~1 범위로 조정

In [3]:
# MNIST 데이터세트
mnist = tf.keras.datasets.mnist

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


## 머신 러닝 모델 빌드

tf.keras.Sequential 모델을 빌드

In [10]:
# Sequential은 각 레이어에 하나의 입력 텐서와 하나의 출력 텐서가 있는 레이어를 쌓는 데 유용
# 이 모델은 Flatten, Dense, Dropout 레이어를 사용

In [5]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Flatten(input_shape=(28, 28)), 			# 입력 레이어
	tf.keras.layers.Dense(128, activation='relu'),			# 은닉 레이어
	tf.keras.layers.Dropout(0.2),
	tf.keras.layers.Dense(10, activation='softmax')
])

d:\01_Programming\100_HugoBank\Mine\study-machine-deep-learning\tf_env\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
predictions = model(X_train[:1]).numpy()
predictions

array([[0.03828733, 0.18386222, 0.09619617, 0.05641898, 0.04280505,
        0.05828954, 0.19838358, 0.10935701, 0.10051107, 0.11588901]],
      dtype=float32)

In [8]:
# tf.nn.softmax 함수는 이러한 로짓을 각 클래스에 대한 확률로 변환
tf.nn.softmax(predictions).numpy()

array([[0.09388327, 0.1085952 , 0.09948045, 0.09560106, 0.09430837,
        0.09578005, 0.11018366, 0.10079835, 0.09991062, 0.10145892]],
      dtype=float32)

In [9]:
# losses.SparseCategoricalCrossentropy를 사용하여 훈련용 손실 함수를 정의
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [10]:
loss_fn(y_train[:1], predictions).numpy()

np.float32(2.3457007)

In [11]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## 모델 훈련 및 평가

In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' ## 초기화할 GPU number

with tf.Graph().as_default():
    # GPU 메모리를 전부 할당하지 않고, 아주 적은 비율만 할당되어 시작됨. 프로세스의 메모리 수요에 따라 자동적으로 증가
	# 단, GPU 메모리를 처음부터 전체 비율을 사용하지 않음
	gpu_options = tf.compat.v1.GPUOptions(allow_growth=True) # V1 ???

In [13]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5


d:\01_Programming\100_HugoBank\Mine\study-machine-deep-learning\tf_env\lib\site-packages\keras\src\backend\tensorflow\nn.py:678: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8660 - loss: 0.4667
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9580 - loss: 0.1452
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9668 - loss: 0.1071
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9743 - loss: 0.0877
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9766 - loss: 0.0732


In [22]:
print('GPU train') # 여기에선 GPU를 사용한다고 무지 빨라지지는 않는다.
with tf.device('/device:GPU:0'):
	model.fit(X_train, y_train, epochs=5)
 
print("CPU Train")
with tf.device('/device:CPU:0'):
	model.fit(X_train, y_train, epochs=5)

GPU train
Epoch 1/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0408 - accuracy: 0.9863
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0386 - accuracy: 0.9868
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0366 - accuracy: 0.9876
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0333 - accuracy: 0.9884
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0322 - accuracy: 0.9892
CPU Train
Epoch 1/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0297 - accuracy: 0.9895
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0287 - accuracy: 0.9905
Epoch 3/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0281 - accuracy: 0.9899
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0246 - accuracy: 0.9914
Epoch 5/5
1875/1875 [==============================] - 13s 7ms

In [14]:
model.evaluate(X_test,  y_test, verbose=2)

313/313 - 1s - 2ms/step - accuracy: 0.9797 - loss: 0.0738


[0.07381288707256317, 0.9797000288963318]

In [15]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [16]:
probability_model(X_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.08544448, 0.08544447, 0.08544496, 0.08618234, 0.08544447,
        0.08544464, 0.08544447, 0.23025526, 0.0854469 , 0.08544807],
       [0.08564731, 0.08781794, 0.22697075, 0.08567169, 0.08564727,
        0.08565592, 0.08564728, 0.08564727, 0.08564735, 0.08564727],
       [0.08534948, 0.23176797, 0.08535661, 0.085351  , 0.08535039,
        0.08534988, 0.08534961, 0.08541638, 0.08535912, 0.0853496 ],
       [0.23187211, 0.08534288, 0.08534452, 0.08534303, 0.08534288,
        0.085345  , 0.08537824, 0.08534547, 0.08534288, 0.08534297],
       [0.08549337, 0.08549134, 0.08549233, 0.0854914 , 0.22950438,
        0.08549143, 0.08549368, 0.08556615, 0.08549371, 0.08648227]],
      dtype=float32)>

### 결론

축하합니다! Keras API를 사용하는 사전에 빌드한 데이터세트를 사용하여 머신 러닝 모델을 훈련했습니다.